# Tune hyperparameters with the Keras Tuner

https://www.tensorflow.org/tutorials/keras/keras_tuner

## Import Keras

In [1]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras

import kerastuner as kt

In [3]:
# Setup memory to fix critical issue
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Dataset Example

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

### Instantiate the tuner and perform hypertuning

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir\intro_to_kt\oracle.json


In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trails=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is {best_hps.get('units')}
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 31s]
val_accuracy: 0.8604166507720947

Best val_accuracy So Far: 0.8809166550636292
Total elapsed time: 00h 06m 25s
INFO:tensorflow:Oracle triggered exit


TypeError: get_best_hyperparameters() got an unexpected keyword argument 'num_trails'

## Checkpoint callback options

In [ ]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)

In [ ]:
ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("After restored, the model accuracy: {acc:5.2f}%".format(acc=100 * acc))

## Save the entire model

In [ ]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
ls saved_model

In [ ]:
ls saved_model\my_model

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

### HDF5 format

In [ ]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))